In [1]:
import pandas as pd
from Bio.SeqUtils import gc_fraction 
import numpy as np
import subprocess
import re

In [2]:
fp = "/home/myoui/shared/ML/Homo_sapiens.GRCh38.ncrna.fa"

In [3]:
data_lists = [[] for _ in range(8)]  # Creates a total of 8 lists for columns one to eight
sequence_list = []
with open(fp, 'r') as file:
    current_sequence = []  
    for line in file:
        line = line.strip()  # Removes whitespace

        # Processes header lines starting with '>'
        if line.startswith('>'):
            # Before processing the new header, appends the current sequence if it exists
            if current_sequence:
                sequence_list.append(''.join(current_sequence))  # Joins and appends sequence
                current_sequence = []  # Resets for the next sequence

            # Splits line into fields
            parts = line.split()

            # Appends to respective lists only if there are at least 8 parts
            for i in range(8):
                if i < len(parts):
                    data_lists[i].append(parts[i].replace('>', '') if i == 0 else parts[i])
                else:
                    data_lists[i].append(None)  # Fills with None if the part doesn't exist

        else:
            # If it's a sequence line (not a header), accumulate the sequence
            current_sequence.append(line)

    # Append the last sequence after the loop
    if current_sequence:
        sequence_list.append(''.join(current_sequence))

# Prints lengths to check for mismatches
for idx, lst in enumerate(data_lists):
    print(f"Column {idx + 1} list length: {len(lst)}")
print(f"Sequence list length: {len(sequence_list)}")

Column 1 list length: 203776
Column 2 list length: 203776
Column 3 list length: 203776
Column 4 list length: 203776
Column 5 list length: 203776
Column 6 list length: 203776
Column 7 list length: 203776
Column 8 list length: 203776
Sequence list length: 203776


In [4]:
# Save the parsed data in dataframe
df = pd.DataFrame(data_lists).T

In [5]:
# Columns for Gene list dataframe
transcript_id = df[0]
chromosome_location = df[2]
ensembl_id = df[3]
gene_type = df[4]
gene_symbol = df[6]

# Removes prefixes
ensembl_id = ensembl_id.str.replace('gene:', '', regex=False)
gene_type = gene_type.str.replace('gene_biotype:', '', regex=False)
gene_symbol = gene_symbol.str.replace('gene_symbol:', '', regex=False)

In [6]:
# Create Gene list dataframe
gene_list = pd.DataFrame({
    'transcript_id' : transcript_id,
    'ensembl_id': ensembl_id,
    'chromosome_location' : chromosome_location,
    'gene_type' : gene_type,
    'gene_symbol' : gene_symbol,
    'sequence' : sequence_list
})

In [7]:
# Save the DataFrame to CSV
output = '/home/myoui/shared/ML/Gene_list.csv'
gene_list.to_csv(output, index=False)

In [8]:
gene_list['gene_type'].unique()

array(['miRNA', 'Mt_tRNA', 'vault_RNA', 'rRNA', 'sRNA', 'scRNA',
       'Mt_rRNA', 'snRNA', 'scaRNA', 'snoRNA', 'ribozyme', 'lncRNA',
       'misc_RNA'], dtype=object)

In [9]:
gene_list['gene_type'].value_counts()

gene_type
lncRNA       196132
misc_RNA       2419
snRNA          2094
miRNA          1945
snoRNA         1020
rRNA             71
scaRNA           51
Mt_tRNA          22
ribozyme          9
sRNA              6
vault_RNA         4
Mt_rRNA           2
scRNA             1
Name: count, dtype: int64

In [10]:
# Categories with significant no. of genes
categories = ['lncRNA', 'misc_RNA', 'snRNA', 'miRNA', 'snoRNA']

In [11]:
# Filter only gene categories except lncRNA
gene_filtered = gene_list[gene_list['gene_type'].isin(categories) & (gene_list['gene_type'] != 'lncRNA')]

In [12]:
# Randomly select 2000 genes with random_state = 42 for reproducibility
lncrna_sample = gene_list[gene_list['gene_type'] == 'lncRNA'].sample(n=2000, random_state=42)

In [13]:
# Add lncRNA
gene_filtered = pd.concat([gene_filtered, lncrna_sample])

In [14]:
# Get GC content and Sequence length
def gc_content(seq):
    return gc_fraction(seq)

def sequence_length(seq):
    return len(seq)

gene_filtered['gc_content'] = gene_filtered['sequence'].apply(gc_content)
gene_filtered['seq_length'] = gene_filtered['sequence'].apply(sequence_length)

In [15]:
# Extract genes above 4000 nucleotides
large_genes = gene_filtered[gene_filtered['seq_length'] > 4000]
large_genes

,transcript_id,ensembl_id,chromosome_location,gene_type,gene_symbol,sequence,gc_content,seq_length
105314,ENST00000717492.1,ENSG00000245694.13,chromosome:GRCh38:16:54905876:54930578:-1,lncRNA,CRNDE,AGACTCTCGAGCGCCGGCCCCAGGATGACAATCACATCCCAGGAGC...,0.409809,19697
123700,ENST00000663612.1,ENSG00000224078.16,chromosome:GRCh38:15:25094037:25122467:1,lncRNA,SNHG14,GGATCGATGATGACTTTTATACATGCATTCCTTGGAAAGCTGAACA...,0.388336,7339
11323,ENST00000659277.1,ENSG00000224924.10,chromosome:GRCh38:21:20735784:20803297:-1,lncRNA,LINC00320,AATCATCTCCCATTGAAAGCTGAATCATGAGGATACCATCTCCAAG...,0.395783,4932
100329,ENST00000522627.1,ENSG00000253311.4,chromosome:GRCh38:5:159776772:159871384:-1,lncRNA,LINC01847,GTTGCATTAAGTCATCGAGTTTGTGGTGCTTTGTTATGGGAGCCTT...,0.414304,5607
38960,ENST00000443641.5,ENSG00000234893.6,scaffold:GRCh38:HSCHR6_MHC_QBL_CTG1:1546099:15...,lncRNA,HCG18,CAATACTCGCCGGAAGCGGAAGCCGCGCCGAGGCTCGTGTAAAAGT...,0.440120,4317
18874,ENST00000415033.4,ENSG00000235903.12,chromosome:GRCh38:13:46052815:46116576:1,lncRNA,CPB2-AS1,CTTCGTGGACACGCCCTCTAGGCGTAAGGTTCCATGCGAGCGGTAA...,0.392840,4078
50616,ENST00000452075.7,ENSG00000293313.2,chromosome:GRCh38:10:42513510:42552794:-1,lncRNA,ZNF37BP,GTTTTTCTCCGGTACAGCCTGGTGAAGGATGCCCGGAGGCCTCGGC...,0.403728,8637
132712,ENST00000624299.1,ENSG00000247809.12,chromosome:GRCh38:15:96285068:96289265:-1,lncRNA,NR2F2-AS1,TTTTCTAATAATTTTTTTTTTTTTTTTTTGAGACAGAGTCTCGCTC...,0.445688,4198
119139,ENST00000668181.1,ENSG00000259234.9,chromosome:GRCh38:15:79187239:79283949:-1,lncRNA,ANKRD34C-AS1,AGTTAGAAGCAGATCAGAAATCGAACACGTTTGCTTTTGCGATTCG...,0.491772,5165
105431,ENST00000711040.1,ENSG00000250337.11,chromosome:GRCh38:5:27472227:27488861:1,lncRNA,PURPL,GGAGCCATGAGGAATTCATGCCTTGCAAAGGGGAGGAGCCTGCCCT...,0.342486,4625


In [16]:
# Remove ENST00000717492.1, ENST00000650366.1 and ENST00000643616.1 from dataframe(large, causes issues with RNAfold)
transcripts_to_remove = ['ENST00000717492.1', 'ENST00000650366.1', 'ENST00000643616.1']
gene_filtered = gene_filtered[~gene_filtered['transcript_id'].isin(transcripts_to_remove)]

In [17]:
# Initialize dictionaries to store structure and mfe
structure_dict = {}
mfe_dict = {}

# Open and parse the RNAfold output file
with open('/home/myoui/shared/ML/rnafold_output.txt', 'r') as f:
    lines = f.readlines()
    
    for i in range(0, len(lines), 3):  # Read the file in chunks of 3 lines
        # Extract transcript ID (remove '>' from the beginning)
        transcript_id = lines[i].strip().replace('>', '')
        
        # Extract structure and MFE from line i+2
        structure_line = lines[i+2].strip()
        structure = structure_line.split(" ")[0]
        
        # Check if MFE is present and can be converted to float
        try:
            mfe = float(structure_line.split(" ")[1].strip("()"))
        except (IndexError, ValueError):
            mfe = None  # Assign None if MFE is missing or can't be converted
        
        # Store structure and mfe in dictionaries
        structure_dict[transcript_id] = structure
        mfe_dict[transcript_id] = mfe

# Add structure and mfe to gene_filtered dataframe by matching transcript_id
gene_filtered['structure'] = gene_filtered['transcript_id'].map(structure_dict)
gene_filtered['mfe'] = gene_filtered['transcript_id'].map(mfe_dict)

In [18]:
gene_filtered['dot_count'] = gene_filtered['structure'].str.count(r'\.')
gene_filtered['bracket_count'] = gene_filtered['structure'].str.count(r'\(') + gene_filtered['structure'].str.count(r'\)')

In [19]:
data = gene_filtered.dropna(subset=['mfe'])


In [20]:
def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

# Generate k-mers and add them as a feature
data['kmers'] = data['sequence'].apply(lambda x: getKmers(x))

# Combine k-mers into a single space-separated string per sequence
data['kmers'] = data['kmers'].apply(lambda x: ' '.join(x))

/tmp/ipykernel_4475/1951693723.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['kmers'] = data['sequence'].apply(lambda x: getKmers(x))
/tmp/ipykernel_4475/1951693723.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['kmers'] = data['kmers'].apply(lambda x: ' '.join(x))


In [21]:
data

,transcript_id,ensembl_id,chromosome_location,gene_type,gene_symbol,sequence,gc_content,seq_length,structure,mfe,dot_count,bracket_count,kmers
0,ENST00000707457.1,ENSG00000291414.1,scaffold:GRCh38:HG1369_PATCH:55354:55416:-1,miRNA,MIR4740,GCCAAGGACTGATCCTCTCGGGCAGGGAGTCAGAGGGGACCGCCCG...,0.682540,63,..((.((((.(((((((((((((.((...((....))..)))))))...,-37.4,17,46,gccaag ccaagg caagga aaggac aggact ggactg gact...
1,ENST00000707458.1,ENSG00000291415.1,scaffold:GRCh38:HG1369_PATCH:107309:107393:-1,miRNA,MIR3186,AGCCTGCGGTTCCAACAGGCGTCTGTCTACGTGGCTTCAACCAAGT...,0.564706,85,(((((..((((((((.((.(((((.(((.((((((((.(((...))...,-44.9,17,68,agcctg gcctgc cctgcg ctgcgg tgcggt gcggtt cggt...
2,ENST00000637028.1,ENSG00000283344.1,scaffold:GRCh38:HG1362_PATCH:80950:81034:1,miRNA,MIR1244-4,ATCTTATTCCGAGCATTCCAGTAACTTTTTTGTGTATGTACTTAGC...,0.317647,85,(((((((.....(((..((((..(((......((((.((((........,-15.0,35,50,atctta tcttat cttatt ttattc tattcc attccg ttcc...
3,ENST00000711167.1,ENSG00000292346.1,chromosome:GRCh38:Y:2609191:2609254:1,miRNA,MIR6089,CCCCGGGCCCGGCGTTCCCTCCCCTTCCGTGCGCCAGTGGAGGCCG...,0.843750,64,(((((..((((.(.(.((((..(((((..((...))..)))))..)...,-34.6,20,44,ccccgg cccggg ccgggc cgggcc gggccc ggcccg gccc...
4,ENST00000711140.1,ENSG00000292355.1,chromosome:GRCh38:Y:1293918:1293992:1,miRNA,MIR3690,CCCATCTCCACCTGGACCCAGCGTAGACAAAGAGGTGTTTCTACTC...,0.533333,75,(((((((.(((.(((..((((.(((((....(((.........)))...,-32.8,25,50,cccatc ccatct catctc atctcc tctcca ctccac tcca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185318,ENST00000662321.1,ENSG00000177788.7,chromosome:GRCh38:1:229267157:229271051:-1,lncRNA,RAB4A-AS1,GGGCGGGGCGCGTGAGGACTTCGGCGCGCGCCGGAAGCACGTGCGC...,0.435714,2520,.(((..((((((((..(....)..))))))))(((((((.(((.((...,-784.5,926,1594,gggcgg ggcggg gcgggg cggggc ggggcg gggcgc ggcg...
119055,ENST00000689196.1,ENSG00000231028.12,chromosome:GRCh38:6:135497944:135528351:1,lncRNA,AHI1-DT,AGGGGGCTCCCGCCTCCCCGGCCACGCGAGCGGGAACAGTCTCCTC...,0.398104,1055,.((((((....))))))..(((((..((((.((((......))))....,-249.8,451,604,aggggg gggggc ggggct gggctc ggctcc gctccc ctcc...
154910,ENST00000820942.1,ENSG00000248538.11,chromosome:GRCh38:8:9151725:9202973:1,lncRNA,PPP1R3B-DT,AGCGCGCCCGGAGTGATCCGGCTGCGGGCCAGGGTCTGGGAAGGGG...,0.499084,1092,.((.((((((((....)))))..))).))((((((((((((.((((...,-335.1,474,618,agcgcg gcgcgc cgcgcc gcgccc cgcccg gcccgg cccg...
72254,ENST00000648393.1,ENSG00000285666.2,chromosome:GRCh38:7:76818377:76903041:-1,lncRNA,description:novel,TTCTGTTTTTTTTTCTCTCCCACCCATCCTTATTGCCAACCACCCC...,0.412121,1155,((((((((.....(((((...((((.(((((.((.(((.......(...,-290.3,457,698,ttctgt tctgtt ctgttt tgtttt gttttt tttttt tttt...


In [22]:
data.to_csv('/home/myoui/shared/ML/gene_list_para.csv', index=False)

In [23]:
df = gene_list
gene_types_of_interest = ['scRNA', 'Mt_rRNA', 'vault_RNA']
filtered_df = df[df['gene_type'].isin(gene_types_of_interest)]
print(filtered_df)
filtered_df.to_csv('/home/myoui/shared/ML/scRNA_Mt_vault.csv', index=False)

          transcript_id         ensembl_id  \
1967  ENST00000363120.1  ENSG00000199990.1   
1968  ENST00000365241.1  ENSG00000202111.1   
1969  ENST00000365645.1  ENSG00000202515.1   
1970  ENST00000602301.2  ENSG00000270123.4   
2048  ENST00000418539.2  ENSG00000236824.2   
2049  ENST00000389680.2  ENSG00000211459.2   
2050  ENST00000387347.2  ENSG00000210082.2   

                             chromosome_location  gene_type gene_symbol  \
1967   chromosome:GRCh38:5:140711275:140711373:1  vault_RNA    VTRNA1-1   
1968   chromosome:GRCh38:5:140718925:140719013:1  vault_RNA    VTRNA1-2   
1969   chromosome:GRCh38:5:140726158:140726246:1  vault_RNA    VTRNA1-3   
1970  chromosome:GRCh38:5:136080470:136080597:-1  vault_RNA    VTRNA2-1   
2048     chromosome:GRCh38:2:47335315:47335514:1      scRNA      BCYRN1   
2049             chromosome:GRCh38:MT:648:1601:1    Mt_rRNA     MT-RNR1   
2050            chromosome:GRCh38:MT:1671:3229:1    Mt_rRNA     MT-RNR2   

                              